In [1]:
import math
import numpy as np 
import pandas as pd
import gradio as gr
import  datetime


In [5]:
import serial.tools.list_ports
import time
ports = serial.tools.list_ports.comports()
for p in ports:
    print(p.device)
print(len(ports), 'ports')

# replace with the correct port and baud rate for your Bluetooth device
port = ports[0].device
baud_rate = 9600

# establish a serial connection
bluetooth = serial.Serial(ports[2].device, baud_rate)
y = [0]

/dev/cu.wlan-debug
/dev/cu.Bluetooth-Incoming-Port
/dev/cu.HC-05
3 ports


In [6]:
def GetVal():
    global y
    #print('reading')
    data = bluetooth.readline()
    #print('done')
    # print the response
    data = data.decode()
    data = data.split(" ")
    #print(data)
    if data[0] == 'CH1':
        data.pop(0)
        #print('c1')
        for val in data:
            try:
                val = float(val)
                if 3.3 > val >0:
                    val = (val)
                    y.append(val)
            except:
                pass
        print(y)


#     print(len(TotalArray))    
#     print((data))
#     time.sleep(2)


In [7]:
x0 = 0
x = np.array([])
    
def UpdateVal():
    print('updating')
    global y,x,x0
    try:
        GetVal()
    except:
        pass
    l = len(y)
    x = np.linspace(x0,x0+l-1,l)
    #print(x,y)



In [15]:
def get_time():
    return datetime.datetime.now()


def get_plot_v1(period_v1=1,xscale_v1 = 1,yscale_v1 = 1):
    global x,y,x0
    #x = np.arange(plot_end_1 - 2 * math.pi, plot_end_1, 0.02)
    #y = np.sin(2 * math.pi * period_v1 * x)
    UpdateVal()
    #y = 0
    vMean = sum(y) / len(y)
    vMean = round(vMean,3)
    vPp = round(max(y)-min(y),3)
    #y = np.float64(0.5)   
    update = gr.LinePlot.update(
        value=pd.DataFrame({"x": x, "y": y}),
        x="x",
        y="y",
        title= f"V mean = {vMean},Vpp = " ,
        width=800,
        height=800,
        y_lim= [ -10,10], x_lim = [x0,x0+2000],
        y_title = 'V'
    )
    if x[-1] > x0+2000:
        x0 += 2000 
        y = [0]
    return update

updating


In [16]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Dropdown(
             ["ON","OFF"], type = "index", multiselect= False, label="On/OFF button")
            button = gr.Textbox(
                "Voltage Channel 1",
                label="",
            )
            period_v1 = gr.Slider(
                label="Period of plot", value=1, minimum=0, maximum=10, step=1
            )
            xscale_v1 = gr.Slider(
                label="xscale", value=1, minimum=0, maximum=10, step=1
            )
            yscale_v1 = gr.Slider(
                label="yscale", value=1, minimum=0, maximum=10, step=1
            )

            plot_v1 = gr.LinePlot(show_label= False )

    demo.load(lambda: datetime.datetime.now(), None, every=1)
    dep = demo.load(get_plot_v1, None, plot_v1, every=1)
    period_v1.change(get_plot_v1, period_v1, plot_v1, every=1, cancels=[dep])
    xscale_v1.change(get_plot_v1, xscale_v1, plot_v1, every=1, cancels=[dep])
    yscale_v1.change(get_plot_v1, yscale_v1, plot_v1, every=1, cancels=[dep])

if __name__ == "__main__":
    demo.queue().launch(debug = True , share = True )




updating
Running on local URL:  http://127.0.0.1:7864
updating
[0, 1.05139, 1.04575, 1.0385, 1.04736, 1.053, 1.05542, 1.05784, 1.04414, 1.04253, 1.04736, 1.05139, 1.05542, 1.05864, 1.04092, 1.0377, 1.04172, 1.04897, 1.05139, 1.05461, 1.05703, 1.0377, 1.04414, 1.04817, 1.05703, 1.05461, 1.05703, 1.04253, 1.04736, 1.053, 1.05703, 1.06025, 1.05784, 1.04414, 1.04656, 1.05623, 1.05542, 1.05864, 1.04817, 1.04656, 1.0522, 1.05381, 1.05784, 1.04897, 1.04736, 1.05703, 1.05381, 1.06187, 1.04172, 1.04897, 1.05381, 1.05461, 1.04575, 1.04817, 1.05461, 1.05784, 1.05703, 1.04495, 1.04897, 1.05381, 1.05703, 1.04092, 1.04333, 1.05059, 1.05461, 1.053, 1.03689, 1.04575, 1.053, 1.04897, 1.03931, 1.04817, 1.05381, 1.05784, 1.0522, 1.04092, 1.04817, 1.05139, 1.05461, 1.05381, 1.04656, 1.03689, 1.04495, 1.04817, 1.05461, 1.05945, 1.05542, 1.04495, 1.0522, 1.05461, 1.05864, 1.06589, 1.04495, 1.05139, 1.04897, 1.05381, 1.04011, 1.04172, 1.05461, 1.053, 1.053, 1.04414, 1.05139, 1.05381, 1.05703, 1.04414, 1.0513

In [ ]:
bluetooth.close()